# Sequence generation

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import pandas as pd
import torch
from evoamp.models import EvoAMP
from evoamp.models._globals import END_TOKEN

torch.manual_seed(0);

With the pretrained model, let's generate a few sequences.

In [2]:
DATA_PATH = "../data/processed/amp_8_35.csv"
MODEL_PATH = "../outputs/2024-05-17/14-53-50/pretrained_model"

In [3]:
model = EvoAMP.load(MODEL_PATH)

In [4]:
df = pd.read_csv(DATA_PATH)

In [5]:
reference_sequence = df.iloc[0]["sequence"]
is_AMP = df.iloc[0]["is_amp"]

print(f"Reference sequence: {reference_sequence}")
print(f"Is AMP: {is_AMP}")

Reference sequence: TWKKGFPHGTCSKCARE
Is AMP: 0


In [6]:
sequences = model.sample(reference_sequence, is_AMP, n_samples=5)

In [7]:
def prettify(sequence: list[list[str]]) -> list[str]:
    start, end = 1, len(sequence) if sequence[-1] != END_TOKEN else len(sequence) - 1
    return "".join(sequence[start:end])


sequences = [prettify(seq) for seq in sequences]

In [8]:
sequences

['IIIKKKKKKKKKKKKKKK',
 'IIIKKKKKKKKKKKKKKK',
 'LKKKKKKKKKKKKKKKKK',
 'GKKKKKKKKKKKKKKKKK',
 'IIIIKKKKKKKKKKKKKK']